# IMPORT MODULES

In [1]:
import sys

sys.path.append('..')

In [2]:
from modules.model import DLMA
from modules.dataset import DataGenerator
import numpy as np
import glob

Using TensorFlow backend.


# IMPORT MODEL

In [3]:
model = DLMA(time_steps=2, classes_num=2)
model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])

W0819 10:06:08.334854 140238473824064 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 10:06:08.348800 140238473824064 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 10:06:08.351204 140238473824064 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 10:06:08.405937 140238473824064 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.place

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2, 128)       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2, 1024)      132096      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 2, 1024)      4096        dense_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 2, 1024)      0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

# IMPORT DATASET

In [ ]:
train_path = 'data/arrays/train/*.npy'
dev_path   = 'data/arrays/dev/*.npy'
test_path  = 'data/arrays/test/*.npy'

partition  = dict()

In [ ]:
train_db = glob.glob(train_path)
dev_db   = glob.glob(dev_path)
test_db  = glob.glob(test_path)

In [ ]:
# Take only the file name, not the full path
partition['train'] = [train_db[i].split('/')[-1] for i in range(len(train_db))]
partition['dev']   = [dev_db[i].split('/')[-1] for i in range(len(dev_db))]
partition['test']  = [test_db[i].split('/')[-1] for i in range(len(test_db))]

In [ ]:
train_generator = DataGenerator(
    partition['train'],
    db_part='train',
    dim=(2, 128),
    batch_size=16,
    n_classes=2,
    shuffle=True
)

dev_generator   = DataGenerator(
    partition['dev'],
    db_part='dev',
    dim=(2, 128),
    batch_size=16,
    n_classes=2,
    shuffle=True
)

test_generator  = DataGenerator(
    partition['test'],
    db_part='test',
    dim=(2, 128),
    batch_size=16,
    n_classes=2,
    shuffle=True
)

# TRAIN

In [ ]:
model.fit_generator(
    generator=train_generator,
    validation_data=dev_generator,
    workers=8,
    epochs=20
)

In [ ]:
model.save_weights('weights_100_epochs.h5')

# TEST MODEL

In [ ]:
model.load_weights('weights_100_epochs.h5')

In [ ]:
model.evaluate_generator(generator=test_generator, verbose=1)

In [ ]:
pred = model.predict_generator(test_generator, verbose=1)
pred = np.argmax(pred, axis=1)